# Generador de instancias
En este notebook está el código para generar los sets de instancias que se usan para experimentar.
- Estas instancias van a ser guardadas en la carpeta __instancias__.
- Cada set estará en su propia carpeta y tendrá un archivo _indice.csv_ que contendrá información sobre las instancias.

In [1]:
import random, math
import pandas as pd
import time
import os

In [2]:
def save_instance(dataset, instance_name, n, aristas):
    with open(F"instancias/{dataset}/{instance_name}.txt", "w") as f:
        print(n, len(aristas), file=f) #escribo la cantidad de nodos seguido de la cantidad de aristas
        for arista in aristas: #para todas las aristas
            print(arista[0], file=f, end= " ") #escribo el primer nodo
            print(arista[1], file=f, end= " ") #el segundo
            print(arista[2], file=f, end =" ") # el peso y salto de linea
            
def save_index(dataset, instances):
    with open(F"instancias/{dataset}/instances.txt", "w") as f:
        for instance in instances: 
            print(instance, file=f)

In [3]:
filas_indice = []
for n in range(3, 103):#creo 100 grafos con n=3....103
    aristas= n*(n-1)/2 #cantidad de aristas de un grafo Kn
    S = []#inicializo la lista donde voy a guardar las aristas
    for i in range(1, n+1):#empiezo en el nodo 1 en vez del nodo 0 por convencion del tp
        for j in range (i+1, n+1):#creo una arista para todo par i, j donde j>i
              S.append([i, j, random.randint(0, 100)]) #la guardo
    save_instance("grafo-propio", F"G-{n}", n, S)
    filas_indice.append(["grafo-propio", F"G-{n}", n, aristas, F"instancias/grafo-propio/G-{n}.txt"])     
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "n", "m", "archivo"]).to_csv("instancias/grafo-propio/indice.csv", index=False, header=True)    

# Peor caso AGM

In [4]:
filas_agm = []
for n in range(3, 103):
    aristas = n*(n-1)/2
    S = []
    for j in range(2,n+1):
        S.append([1, j, 1]) #quiero que el AGM sea una estrella centrada en V1
    for i in range(2, n+1): #el resto
        if i+1<n+1:
            S.append([i,i+1, random.randint(150, 200)]) #la arista hacia el proximo en dfs es MUY costosa
        for j in range(i+2, n+1):
            S.append([i,j, random.randint(20,100)]) #el resto es malo, pero no tanto como las que elije el dfs
    save_instance("peor-caso-agm", F"AGM-{n}",n,S)
    filas_agm.append(["peor-caso-agm",F"AGM-{n}",n,aristas,F"instancias/peor-caso-agm/AGM-{n}.txt"])
pd.DataFrame(filas_agm, columns=["dataset", "instancia", "n", "m", "archivo"]).to_csv("instancias/peor-caso-agm/indice.csv", index=False, header=True)

# Cargamos los grafos para los cuales conocemos los resultados optimos

In [6]:
filas_indice=[]
archivos = os.listdir("instancias/descargas/data")
for instancia in archivos:
    n = ""
    for letra in instancia:
        if letra.isnumeric():
            n = n+letra
    n = int(n)
    aristas= n*(n-1)/2 #cantidad de aristas de un grafo Kn
    nombre = instancia.split(".")[0]
    soluciones = open("instancias/descargas/pesos_optimos.txt", "r")
    encontre = False
    peso_opt = 0
    print(nombre)
    while(not encontre):
        linea = soluciones.readline().split(":")
        if nombre == linea[0]:
            encontre = True
            peso_opt = int(linea[1])
        if "FIN" == linea[0]:
            break
    if peso_opt == 0:
            print("falta el optimo del caso " + nombre)
            time.sleep(5)
    filas_indice.append(["sol_optimas", nombre, n, aristas, F"instancias/descargas/data/{nombre}.tsp", peso_opt])
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "n", "m", "archivo", "peso_optimo"]).to_csv("instancias/descargas/indice.csv", index=False, header=True)
    


u1432
gr666
rl1304
pr107
rat99
d2103
kroA200
rl1323
p654
u574
pr152
bier127
pcb1173
bays29
u1060
pr76
fl1577
kroB100
u2152
ch150
pr264
gr202
att532
dantzig42
kroB150
eil76
ts225
kroB200
pr144
u724
pa561
linhp318
u2319
pr2392
dsj1000
kroA150
kroA100
burma14
u159
d1655
gr229
gr137
kroC100
ch130
pr136
a280
eil51
lin105
rat195
gil262
d657
bayg29
gr431
fl417
pr299
pr1002
tsp225
nrw1379
ulysses16
vm1748
eil101
vm1084
rl1889
u1817
pr124
berlin52
d1291
d198
pr439
st70
pcb442
gr96
rat783
gr120
pr226
rd400
kroD100
fl1400
ali535
lin318
rat575
kroE100
att48
ulysses22
d493
rd100


# Peor Caso GA 

In [13]:
filas_indice = []
for n in range(3, 103):#creo 100 grafos con n=3....103
    aristas= n*(n-1)/2 #cantidad de aristas de un grafo Kn
    S = []#inicializo la lista donde voy a guardar las aristas
    orden = [] #inicializo el vector que tendrá el orden del ciclo malo para GA
    
    for i in range (1,n+1):
        orden.append(i) #lleno con los vértices
    elementos_orden = orden #copio para hacer un shuffle
    random.shuffle(elementos_orden) #creo un orden random para los vértices y hacer un ciclo malo para GA
    print(elementos_orden)
    for i in range(0,n):
        orden[elementos_orden[i]-1] = i+1 #escribo en orden para que el indice sea el vértice y el elemento sea el orden en el ciclo 
    
    for i in range(1, n+1):#empiezo en el nodo 1 en vez del nodo 0 por convencion del tp
        for j in range (i+1, n+1):#creo una arista para todo par i, j donde j>i
            if orden[i-1] - orden[j-1] == 1 | orden[j-1] - orden[i-1] == 1: #si los vértices son consecutivos segun el orden
                S.append([i,j, random.randint(1,49)]) #guardo una arista pequeña
            elif (orden[i-1] == 1 & orden[j-1] == n) | (orden[j-1] == 1 & orden[i-1] == n): #si estamos hablando de la utima conexion del ciclo
                S.append([i,j, random.randint(n*100, n*500)]) #guardo una arista inutilizable
            else: #si los vértices no son consecutivos en orden
                S.append([i, j, random.randint(50, 100)]) #uso una arista de peso medio
    save_instance("peor-caso-ga", F"G-{n}", n, S)
    filas_indice.append(["peor-caso-ga", F"G-{n}", n, aristas, F"instancias/peor-caso-ga/G-{n}.txt"])     
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "n", "m", "archivo"]).to_csv("instancias/peor-caso-ga/indice.csv", index=False, header=True)    

[3, 2, 1]
[3, 2, 4, 1]
[2, 4, 3, 5, 1]
[6, 5, 4, 1, 2, 3]
[2, 3, 7, 4, 5, 1, 6]
[5, 7, 4, 1, 3, 6, 2, 8]
[1, 7, 2, 4, 8, 5, 3, 9, 6]
[7, 2, 9, 10, 6, 3, 5, 1, 8, 4]
[7, 10, 4, 1, 5, 3, 9, 11, 6, 8, 2]
[7, 11, 8, 1, 9, 5, 10, 2, 3, 12, 6, 4]
[6, 7, 3, 13, 2, 4, 12, 1, 10, 9, 11, 5, 8]
[9, 12, 10, 3, 4, 7, 11, 8, 13, 6, 14, 1, 2, 5]
[14, 1, 15, 9, 12, 10, 13, 3, 6, 8, 4, 5, 11, 7, 2]
[6, 9, 13, 1, 14, 8, 10, 5, 7, 11, 12, 4, 3, 16, 15, 2]
[7, 11, 9, 4, 13, 5, 16, 3, 12, 8, 14, 6, 17, 2, 15, 10, 1]
[2, 12, 11, 9, 7, 5, 6, 14, 1, 16, 13, 15, 18, 3, 10, 8, 17, 4]
[2, 10, 3, 1, 16, 4, 6, 9, 7, 14, 18, 11, 8, 5, 12, 17, 15, 19, 13]
[1, 11, 8, 4, 20, 14, 9, 16, 13, 2, 17, 10, 7, 5, 19, 15, 3, 18, 12, 6]
[9, 13, 8, 18, 14, 10, 20, 6, 21, 16, 12, 7, 19, 17, 5, 15, 1, 2, 11, 4, 3]
[19, 14, 18, 13, 9, 15, 7, 12, 3, 10, 21, 5, 11, 1, 8, 20, 6, 2, 17, 16, 4, 22]
[19, 4, 21, 16, 8, 11, 22, 10, 23, 17, 20, 9, 15, 5, 13, 2, 7, 14, 12, 6, 18, 1, 3]
[13, 2, 19, 6, 14, 7, 22, 1, 12, 16, 8, 11, 4, 15, 10, 

[16, 66, 54, 53, 25, 8, 30, 45, 67, 22, 27, 35, 4, 40, 15, 5, 55, 24, 39, 65, 64, 13, 20, 3, 6, 63, 48, 29, 32, 62, 11, 38, 56, 33, 46, 18, 44, 9, 7, 1, 12, 31, 68, 69, 17, 61, 41, 58, 59, 37, 52, 10, 57, 60, 50, 47, 49, 21, 36, 43, 51, 28, 42, 26, 19, 14, 2, 23, 34]
[50, 43, 40, 7, 9, 8, 57, 30, 11, 54, 25, 12, 23, 61, 14, 42, 34, 53, 46, 58, 65, 56, 37, 35, 15, 48, 52, 39, 2, 45, 26, 64, 62, 22, 6, 19, 27, 44, 55, 33, 59, 24, 49, 63, 1, 13, 10, 47, 36, 41, 67, 4, 21, 31, 32, 29, 5, 68, 20, 51, 16, 69, 66, 38, 60, 70, 28, 17, 3, 18]
[19, 49, 6, 45, 3, 67, 59, 56, 47, 62, 68, 53, 36, 61, 57, 29, 24, 63, 54, 35, 46, 51, 65, 60, 21, 28, 8, 39, 12, 58, 4, 13, 9, 48, 26, 34, 31, 16, 70, 23, 41, 38, 14, 37, 32, 17, 2, 1, 30, 22, 5, 52, 7, 50, 15, 20, 69, 25, 11, 42, 10, 43, 66, 18, 27, 55, 44, 33, 71, 64, 40]
[62, 49, 56, 36, 38, 68, 69, 63, 9, 61, 10, 54, 26, 23, 43, 64, 40, 32, 60, 8, 17, 16, 72, 2, 25, 30, 70, 3, 53, 57, 29, 65, 58, 18, 52, 44, 51, 21, 55, 11, 6, 50, 5, 7, 45, 47, 24, 46

[15, 63, 6, 18, 43, 76, 38, 5, 55, 8, 81, 92, 34, 50, 75, 11, 67, 59, 2, 32, 31, 35, 66, 13, 94, 77, 88, 20, 26, 4, 47, 25, 7, 62, 82, 61, 87, 74, 46, 56, 29, 39, 33, 42, 68, 65, 93, 23, 80, 10, 89, 54, 48, 69, 40, 84, 16, 24, 51, 30, 72, 36, 58, 3, 83, 17, 44, 86, 85, 37, 70, 19, 28, 41, 1, 57, 12, 49, 91, 64, 53, 73, 79, 45, 60, 52, 14, 78, 22, 9, 27, 21, 90, 71]
[75, 42, 78, 45, 24, 22, 18, 17, 63, 56, 62, 74, 90, 9, 35, 66, 79, 33, 91, 47, 54, 32, 83, 4, 12, 49, 80, 1, 34, 86, 65, 13, 70, 82, 2, 7, 60, 40, 51, 61, 31, 84, 68, 64, 16, 14, 20, 5, 59, 46, 37, 28, 72, 48, 85, 36, 93, 76, 8, 71, 3, 81, 29, 58, 10, 6, 92, 87, 25, 57, 39, 44, 77, 11, 89, 50, 21, 41, 15, 26, 69, 88, 23, 53, 55, 67, 38, 52, 27, 73, 95, 94, 43, 30, 19]
[59, 10, 89, 54, 84, 46, 75, 71, 45, 33, 3, 72, 90, 22, 42, 34, 18, 19, 91, 30, 65, 78, 81, 25, 32, 1, 83, 93, 15, 9, 64, 61, 53, 8, 56, 24, 41, 88, 43, 94, 48, 70, 47, 57, 74, 96, 73, 52, 68, 76, 5, 28, 36, 2, 11, 27, 12, 62, 82, 58, 66, 14, 37, 85, 92, 26, 6

In [ ]:
#agrego comentario para pushear
